In [16]:
import boto3, os

In [17]:
ec2 = boto3.resource('ec2')
route53 = boto3.client('route53')

In [23]:
# Delete the DNS record for the basic instance
dns_name = 'doug-nicholson.net'
record_name = 'basic.' + dns_name
response = route53.list_hosted_zones_by_name(
    DNSName=dns_name
)
zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.list_resource_record_sets(
    HostedZoneId=zone_id,
    StartRecordName=record_name,
    MaxItems='1'
)
resource_record_sets = response['ResourceRecordSets']
if len(resource_record_sets) > 0:
    resource_record_set = resource_record_sets[0]
    if resource_record_set['Name'] == record_name + '.':
        route53_waiter = route53.get_waiter('resource_record_sets_changed')
        response = route53.change_resource_record_sets(
            HostedZoneId=zone_id,
            ChangeBatch={
                'Changes': [
                    {
                        'Action': 'DELETE',
                        'ResourceRecordSet': resource_record_set
                    }
                ]
            }
        )
        route53_waiter.wait(
            Id=response['ChangeInfo']['Id'][8:]
        )

In [ ]:
# Remove the entry for this host in the known_hosts file
with open('/Users/dougjam/.ssh/known_hosts', 'r') as f:
    lines = [line for line in f]
    f.close()
with open('/Users/dougjam/.ssh/known_hosts', 'w') as f:
    for line in lines:
        if line.find('basic.doug-nicholson.net') == -1:
            f.write(line)
    f.close()

In [28]:
# Delete any instances that were started
key_pair_name = 'basic-launch-key-pair'
instance_iterator = ec2.instances.filter(
    Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_pair_name
            ]
        }
    ]
)
try:
    instance_list = list(instance_iterator)
except:
    pass
else:
    for instance in instance_list:
        instance.terminate()
    for instance in instance_list:
        instance.wait_until_terminated()

In [29]:
# Delete the key pair and the key pair file if they exist
key_pair_info_iterator = ec2.key_pairs.filter(
    KeyNames=[
        key_pair_name
    ]
)
try:
    # If this causes an exception, nothing needs to be done
    key_pair_info_list = list(key_pair_info_iterator)
except:
    pass
else:
    key_pair_info_list[0].delete()
key_pair_file_name = '/Users/dougjam/.ssh/' + key_pair_name + '.pem'
if os.path.isfile(key_pair_file_name):
    os.remove(key_pair_file_name)